# L5c: A Deeper Dive into Flux Balance Analysis Constraints
In this lecture, we will continue our disucssion of flux balance analysis, and will explore the what we can do with the material balance and flux bounds constraints. However be before we do that, let's review the key concepts we'll talk about today and the linear programming problem that we will solve.

* __Flux balance analysis (FBA)__ is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where production, consumption and transport rates of metabolites are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. 
* __Material balance constraints__ are used to ensure that the flow of metabolites into and out of a system is balanced (and physical). These constraints are typically represented as a set of linear equations, where the coefficients represent the stoichiometry of the reactions in the system, in combination with transport and dilution terms.
* __Flux bounds constraints__ limit the range of possible fluxes through a metabolic network. These bounds can be used to incorporate additional information into the FBA problem, such as experimental data or prior knowledge about the system.


Lecture notes can be downloaded: [here!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/Notes.pdf)

## Flux Balance Analysis Problem Formulation
Flux balance analysis (FBA) is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where production, consumption and transport rates of metabolites are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. The linear program is:
$$
\begin{align*}
\max_{\hat{v}}\quad&  \sum_{i\in\mathcal{R}}c_{i}\hat{v}_{i}\\
\text{subject to}\quad & \sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s} + \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V = \frac{d}{dt}\left(C_{i}V\right)\qquad\forall{i\in\mathcal{M}}\\
& \mathcal{L}_{j}\leq\hat{v}_{j}\leq\mathcal{U}_{j}\qquad\forall{j\in\mathcal{R}}
\end{align*}
$$
Here, $\sigma_{ij}$ are elements of the stoichiometric matrix $\mathbf{S}$, $c_{i}$ are objective coefficients (you choose), $\hat{v}_{i}$ are unknown fluxes, and $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ are the lower and upper flux bounds, respectively.

## A model for flux bounds
The flux bounds are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric array. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the flux estimation problem. A general model for these bounds is given by:
$$
\begin{align*}
-\delta_{j}\underbrace{\left[{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}\right]}_{\text{reverse: other functions or parameters?}}\leq\hat{v}_{j}\leq{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}
\end{align*}
$$
where $V_{max,j}^{\circ}$ denotes the maximum reaction velocity (units: `flux`) computed at some _characteristic enzyme abundance_. Thus, the maximum reaction velocity is given by:
$$
V_{max,j}^{\circ} = k_{cat,j}^{\circ}e^{\circ}
$$
where $k_{cat,j}$ is the catalytic constant or turnover number for the enzyme (units: `1/time`) and $e^{\circ}$ is a characteristic enzyme abundance (units: `concentration`). The term $\left(e/e^{\circ}\right)$ is a correction to account for the _actual_ enzyme abundance catalyzing the reaction (units: `dimensioness`). The $\theta_{j}\left(\dots\right)\in\left[0,1\right]$ is the current fraction of maximial enzyme activity of enzyme $e$ in reaction $j$. The activity model $\theta_{j}\left(\dots\right)$ describes [allosteric effects](https://en.wikipedia.org/wiki/Allosteric_regulation) on the reaction rate, and is a function of the regulatory and the chemical state of the system, the concentration of substrates, products, and cofactors (units: `dimensionless`).
Finally, the $f_{j}\left(\dots\right)$ is a function describing the substrate (reactants) dependence of the reaction rate $j$ (units: `dimensionless`). 

* __Parameters__: We need estimates for the $k_{cat,j}^{\circ}$ for all enzymes in the system we are interested in, and a _reasonable policy_ for specifying a charactertic value for $e^{\circ}$. In addition, the $\theta_{j}\left(\dots\right)$ and $f_{j}\left(\dots\right)$ models can also have associated parameters, e.g., saturation or binding constants, etc. Thus, we need to estimate these from literature studies or experimental data.
* __Reversibility__: Next, we need to estimate the binary direction parameter $\delta_{j}\in\left\{0,1\right\}$. The value of $\delta_{j}$ describes the reversibility of reaction $j$; if reaction $j$ is __reversible__ $\delta_{j}=1$. If reaction $j$ is __irreversible__ $\delta_{j}=0$

### Simplfied bounds model
Let's initially assume that $(e/e^{\circ})\sim{1}$, there are no allosteric inputs $\theta_{j}\left(\dots\right)\sim{1}$, and the substrates are saturating $f_{j}\left(\dots\right)\sim{1}$. 
Then, the flux bounds are given by:
$$
\begin{align*}
-\delta_{j}V_{max,j}^{\circ}\leq{\hat{v}_{j}}\leq{V_{max,j}^{\circ}}
\end{align*}
$$
This is a simple model for the flux bounds, and it is easy to see that the flux bounds are a function of the maximum reaction velocity, which is a function of the catalytic constant or turnover number and our assumed value of a characteristic enzyme abundance.

The turnover number, $k_{cat}$, measures an enzyme's catalytic activity, defined as the number of substrate molecules converted to product per enzyme molecule per unit time. Units are typically `1/time` (e.g., `1/s` or `1/min`). Values can be obtained from primary literature or databases like [BRENDA](https://www.brenda-enzymes.org/):

* [Antje Chang et al., BRENDA, the ELIXIR core data resource in 2021: new developments and updates, Nucleic Acids Research, Volume 49, Issue D1, 8 January 2021, Pages D498–D508, https://doi.org/10.1093/nar/gkaa1025](https://academic.oup.com/nar/article/49/D1/D498/5992283)

Use [BRENDA](https://www.brenda-enzymes.org/) to find turnover numbers for:

* __Enzyme 1__: Arginase (EC 3.5.3.1) in humans.
* __Enzyme 2__: Argininosuccinate synthase (EC 6.3.4.5) in humans.

### eQuilibrator
The reversibility parameter $\delta_{j}$ can be computed in one of several possible ways. For example, one method in the literature is to use the sign of Gibbs reaction energy:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }\text{sign}\left(\Delta{G}^{\circ}\right)<0\quad\text{irreversible} \\
1 & \text{if }\text{sign}\left(\Delta{G}^{\circ}\right)\geq{0}\quad\text{reversible}
\end{cases}
\end{equation*}
$$
Alternatively, the value of $\delta_{j}$ can be assigned based upon a cutoff $K^{\star}$ on the equilibrium constant:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }K_{eq}>\,K^{\star}\quad\text{irreversible} \\
1 & \text{if }K_{eq}\leq\,K^{\star}\quad\text{reversible}
\end{cases}
\end{equation*}
$$
where you specify the value $K^{\star}$ based upon some intution or other criteria. We can compute the $\Delta{G}^{\circ}$ values using [eQuilibrator](https://equilibrator.weizmann.ac.il):

* [Beber ME, Gollub MG, Mozaffari D, Shebek KM, Flamholz AI, Milo R, Noor E. eQuilibrator 3.0: a database solution for thermodynamic constant estimation. Nucleic Acids Res. 2022 Jan 7;50(D1): D603-D609. doi: 10.1093/nar/gkab1106. PMID: 34850162; PMCID: PMC8728285.](https://pubmed.ncbi.nlm.nih.gov/34850162/)

The [eQuilibrator application programming interface](https://equilibrator.weizmann.ac.il) is a tool for thermodynamic calculations in biological reaction networks. It was developed by the [Milo lab](https://www.weizmann.ac.il/plants/Milo/) at the Weizmann Institute in Rehovot, Israel. The [`eQuilibrator.jl` package](https://github.com/stelmo/eQuilibrator.jl) is a [Julia](https://julialang.org) wrapper around eQuilibrator (which is written in Python). 